# COPA Demo

In [4]:
if not "cwd" in globals():
    cwd = Path(os.getcwd())
sys.path.insert(0, str(cwd.parents[0]))

In [5]:
TASK_NAME = "COPA"
AUX_TASK_NAME = "SWAG"
BERT_MODEL = "bert-large-cased"

dataloader_config = {
    "batch_size": 4,
    "data_dir": os.environ["SUPERGLUEDATA"],
    "splits": ["train", "valid"],
}

trainer_config = {
    "lr": 1e-5,
    "optimizer": "adam",
    "n_epochs": 10,
    "conter_unit": "epochs",
    "evaluation_freq": 0.25,
}

### Train Primary Task

In [6]:
from dataloaders import get_dataloaders

# Loading primary task data
copa_dataloaders = get_dataloaders(
    task_name=TASK_NAME,
    tokenizer_name=BERT_MODEL,
    **dataloader_config
)

In [7]:
from superglue_tasks import task_funcs

# Defining task
copa_task = task_funcs[TASK_NAME](BERT_MODEL)

In [8]:
from snorkel.mtl.model import MultitaskModel

# Defining model
copa_model = MultitaskModel(tasks=[copa_task])

In [9]:
from snorkel.mtl.trainer import Trainer

# Defining trainer object
trainer = Trainer(**trainer_config)

In [10]:
# Training model -- takes a long time on CPU!

# trainer.train_model(copa_model, copa_dataloaders)

In [11]:
# Loading weights from model trained as in above cell
copa_model.load('best_model_COPA_SuperGLUE_valid_accuracy.pth')

# Evaluating model
copa_train_loader, copa_dev_loader = copa_dataloaders
scratch_score = copa_model.score(copa_dev_loader)
print(f"COPA Accuracy Using Random Initialization: {scratch_score['COPA/SuperGLUE/valid/accuracy']}")

COPA Accuracy Using Random Initialization: 0.72


### Train model for auxiliary task

In [12]:
# Getting dataloaders, task, and model for SWAG

swag_dataloaders = get_dataloaders(
    task_name=AUX_TASK_NAME,
    tokenizer_name=BERT_MODEL,
    **dataloader_config
)

swag_task = task_funcs[AUX_TASK_NAME](BERT_MODEL)
swag_model = MultitaskModel(tasks=[swag_task])

In [13]:
# Training on SWAG and saving model -- takes a long time on CPU!
from snorkel.mtl.trainer import Trainer

trainer = Trainer(**trainer_config)
# trainer.train_model(swag_model, swag_dataloaders)
# swag_model.save('best_model_SWAG_SuperGLUE.pth')

### Fine-tune double pre-trained model (BERT + SWAG)

In [14]:
from copy import deepcopy

# Loading SWAG-pretrained weights into COPA model
copa_swag_model = deepcopy(copa_model)

# Commenting to save time
# copa_swag_model.load('best_model_SWAG_SuperGLUE.pth')

In [ ]:
from snorkel.mtl.trainer import Trainer

# Train model using SWAG pretrained initialization
trainer = Trainer(**trainer_config)

# Commenting training to save time!
# trainer.train_model(copa_swag_model, copa_dataloaders)

# Loading instead!
pretrained_swag_model_path = 'best_model_COPA_SWAG_AUX_SuperGLUE_valid_accuracy.pth'
copa_swag_model.load(pretrained_swag_model_path)

In [ ]:
pretrained_score = copa_swag_model.score(copa_dev_loader)

### Compare pretrained to non-pretrained model

In [ ]:
print(f"COPA Accuracy Using SWAG Pretrained Weights: {pretrained_score['COPA/SuperGLUE/valid/accuracy']}")
print(f"COPA Accuracy Using Random Initialization: {scratch_score['COPA/SuperGLUE/valid/accuracy']}")